In [2]:
import requests
import warnings
#from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import unittest
import tables
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.io as pio
import math

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [119]:
pio.templates.default = "plotly_dark"

In [213]:
ticker = yf.Ticker("AAPL")
cash_flow_stmnt = ((ticker.cash_flow).T)
income_stmnt = (ticker.income_stmt).T
balance_sheet_stmnt = (ticker.balance_sheet).T
info = ticker.info

In [185]:
industry = info.get('industry')
sector = info.get('sector')
description = info.get('longBusinessSummary')
beta = info.get('beta')
sharesOutstanding =  info.get('sharesOutstanding')
name = info.get('longName')
currentPrice = info.get('currentPrice')
targetHighPrice = info.get('targetHighPrice')
targetLowPrice = info.get('targetLowPrice')
targetMeanPrice = info.get('targetMeanPrice')


In [186]:
cash_flow_stmnt

,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Issuance Of Capital Stock,Capital Expenditure,Interest Paid Supplemental Data,Income Tax Paid Supplemental Data,End Cash Position,Beginning Cash Position,...,Change In Inventory,Change In Receivables,Changes In Account Receivables,Other Non Cash Items,Stock Based Compensation,Deferred Tax,Deferred Income Tax,Depreciation Amortization Depletion,Depreciation And Amortization,Net Income From Continuing Operations
2024-09-30,108807000000.0,-94949000000.0,-9958000000.0,0.0,NaN,-9447000000.0,NaN,26102000000.0,29943000000.0,30737000000.0,...,-1046000000.0,-5144000000.0,-3788000000.0,-2266000000.0,11688000000.0,NaN,NaN,11445000000.0,11445000000.0,93736000000.0
2023-09-30,99584000000.0,-77550000000.0,-11151000000.0,5228000000.0,NaN,-10959000000.0,3803000000.0,18679000000.0,30737000000.0,24977000000.0,...,-1618000000.0,-417000000.0,-1688000000.0,-2227000000.0,10833000000.0,NaN,NaN,11519000000.0,11519000000.0,96995000000.0
2022-09-30,111443000000.0,-89402000000.0,-9543000000.0,5465000000.0,NaN,-10708000000.0,2865000000.0,19573000000.0,24977000000.0,35929000000.0,...,1484000000.0,-9343000000.0,-1823000000.0,1006000000.0,9038000000.0,895000000.0,895000000.0,11104000000.0,11104000000.0,99803000000.0
2021-09-30,92953000000.0,-85971000000.0,-8750000000.0,20393000000.0,1105000000.0,-11085000000.0,2687000000.0,25385000000.0,35929000000.0,39789000000.0,...,-2642000000.0,-14028000000.0,-10125000000.0,-4921000000.0,7906000000.0,-4774000000.0,-4774000000.0,11284000000.0,11284000000.0,94680000000.0
2020-09-30,NaN,NaN,NaN,NaN,880000000.0,NaN,3002000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-215000000.0,-215000000.0,NaN,NaN,NaN


In [188]:
metric_name = 'Free Cash Flow'
metric_no_na = cash_flow_stmnt[metric_name]
metric_no_na = metric_no_na.dropna()
metric_no_na = metric_no_na.iloc[::-1]
metric_growth = metric_no_na.pct_change(fill_method=None)
metric_growth = metric_growth.iloc[::-1]
metric_growth=metric_growth.astype(float)
metric_growth = metric_growth.round(decimals=2)
metric_growth = metric_growth.astype('string') + " %"
meric_ot_fig = go.Figure()
meric_ot_fig.add_traces(go.Bar(x = cash_flow_stmnt.index,y = cash_flow_stmnt[metric_name], text=metric_growth, marker_color='salmon'))
meric_ot_fig.update_xaxes(title_text = 'Date')
meric_ot_fig.update_yaxes(title_text = metric_name)
meric_ot_fig.update_layout(title_text=f'{metric_name} Over Time')

In [189]:
prices = yf.download(tickers=['AAPL'], start='2020-01-01')['Close']
prices = prices.reset_index()
stock_price = go.Figure()
stock_price.add_trace(go.Scatter(x = prices['Date'], y=prices['Close']))
stock_price.update_xaxes(title_text = 'Date')
stock_price.update_yaxes(title_text = 'Price')
stock_price.update_layout(title_text='Stock Price Over Time')

[*********************100%%**********************]  1 of 1 completed


In [153]:
cash_flow_growth_rates = [5,5,3,2,1]
curr_FCF = cash_flow_stmnt['Free Cash Flow'].to_list()[0]
curr_FCF

11807000000.0

{'Free Cash Flow Current': 11807000000.0}

In [221]:
curr_FCF = cash_flow_stmnt['Free Cash Flow'].to_list()[0]
future_Cash_Flows = {'Free Cash Flow Current': cash_flow_stmnt['Free Cash Flow'].to_list()[0]
        }
temp_fcf = curr_FCF
for i in range(0,len(cash_flow_growth_rates)):
    temp_fcf = temp_fcf+((cash_flow_growth_rates[i]/100)*temp_fcf)
    future_Cash_Flows[f'Forecast {i+1}'] =temp_fcf
future_Cash_Flows

{'Free Cash Flow Current': 108807000000.0,
 'Forecast 1': 114247350000.0,
 'Forecast 2': 119959717500.0,
 'Forecast 3': 123558509025.0,
 'Forecast 4': 126029679205.5,
 'Forecast 5': 127289975997.555}

In [226]:
future_cash_Flows_list = list(future_Cash_Flows.values())
future_cash_Flows_list

[108807000000.0,
 114247350000.0,
 119959717500.0,
 123558509025.0,
 126029679205.5,
 127289975997.555]

In [241]:
wacc = 0.05
g = 0.02

In [237]:
tv = (future_cash_Flows_list[-1]*g)/(wacc-g)
tv

458702616.2074053

In [239]:
future_cash_Flows_list = [100,110,125,140]

In [ ]:
pv = 0
for t in range(1,len(future_cash_Flows_list)):
    print(future_cash_Flows_list[t])
    pv = pv + future_cash_Flows_list[t]/(1+wacc)**t
#pv = pv+(tv/(1+wacc)**(len(future_cash_Flows_list)+1))
pv

110
125
140


339.0778533635676

In [12]:
market_index = "^SP500TR"
market_data = yf.Ticker(market_index).history(period="10y")
market_return = (market_data["Close"].iloc[-1] / market_data["Close"].iloc[0]) ** (1/10) - 1
market_return

0.1361278526332117